## 1. pySpark DataFrame

In [ ]:
!pip install pyspark
!curl -L -o "/usr/local/lib/python3.6/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar" http://dl.bintray.com/spark-packages/maven/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
!apt-get install openjdk-8-jdk-headless -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 417, in run
    _, build_failures = build(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 320, in build
    wheel_file = _build_one(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 194, in _build_one
    wheel_path = _build_one_inside_env(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 241, in _build_one_inside_en

In [ ]:
import os
import numpy as np
import pandas as pd
import nltk
import pyspark
import pyspark.sql.functions as F
import sys
import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from datetime import datetime
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from pyspark import SparkContext, SparkConf
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import col, sum as pyspark_sum

ModuleNotFoundError: ignored

In [ ]:
os.environ["JAVA_HOME"]= "/usr/lib/jvm/java-8-openjdk-amd64"

sc= pyspark.SparkContext()
spark= SparkSession.builder.getOrCreate()

In [ ]:
path= "/content/drive/MyDrive/23 U 4-2/빅데이터처리및응용/Data/"

movie_sample= pd.read_csv(path+ "1206 sample_data_pp.csv")
movie_sample.head()

In [ ]:
def parse_list(s):
  return [float(x) for x in s.strip('][').split(',')]

columns_to_convert = ['title', 'era', 'genres', 'tag', 'storyline', 'runtime', 'continent', 'age_rating']
for col in columns_to_convert:
  movie_sample[col] = movie_sample[col].apply(parse_list)

movie_sample.head()

In [ ]:
schema= [
    StructField("movieId", StringType(), False),
    StructField("title", ArrayType(FloatType()), False),
    StructField("genres", ArrayType(FloatType()), True),
    StructField("storyline", ArrayType(FloatType()), True),
    StructField("country", ArrayType(FloatType()), True),
    StructField("release_year", ArrayType(FloatType()), True),
    StructField("runtime", ArrayType(FloatType()), True),
    StructField("age_rating", ArrayType(FloatType()), True),
    StructField("tag", ArrayType(FloatType()), True)
]

spMovies= spark.createDataFrame(movie_sample, schema= StructType(schema))
spMovies.show()

## 2. pandas DataFrame

In [1]:
import os
import numpy as np
import pandas as pd
import nltk
import sys
import warnings
warnings.filterwarnings('ignore')

from scipy.spatial.distance import cosine
from collections import Counter
from datetime import datetime
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
path= "/content/drive/MyDrive/23 U 4-2/빅데이터처리및응용/Data/"

movie_sample= pd.read_csv(path+ "1210 all_pp.csv")
movie_sample.head()

,movieId,title,era,genres,tag,storyline,runtime,continent,age_rating
0,1,"[-0.014765826907629767,-0.0001763248195250829,...","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...","[-0.06418165498180316,-0.009971046587452292,-0...","[0.019509797799400985,-0.022897314243508795,-0...",[-0.8024960503192684],"[0, 0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]"
1,2,"[-0.020792629569768906,-0.005490744020789862,-...","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[-0.025203625322319567,-0.023397489140431084,0...","[0.015569244019747148,-0.0014084530279929327,0...",[-0.04949848793283062],"[0, 0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]"
2,3,"[-0.0010529104038141668,-0.003023075812961906,...","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.02361708382765452,-0.006075011993137498,0....","[0.0034510755504015833,-0.013722939333092654,-...",[-0.14771556128758348],"[0, 0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]"
3,4,"[-0.003295399248600006,-0.007947875885292888,-...","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.008344948175363243,-0.024232767755165696,0...","[0.005609218422950317,-0.022722859384838298,0....",[0.6052820010988548],"[0, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 0]"
4,5,"[0.007662885689309664,-0.004842474711559979,-0...","[0, 0, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.013301791623234749,-0.008611441900332768,0...","[0.004049535797726687,-0.010654402755038893,-0...",[0.015979560970337654],"[0, 0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]"


## 3. pandas Cosine-Similarity

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
import ast

### 3.1 Cosine - All

In [4]:
movie_sample= pd.read_csv(path+ "1210 all_pp.csv")

In [ ]:
# Cosine Similarity
feature_columns = ['title', 'era', 'genres', 'tag', 'storyline', 'runtime', 'continent', 'age_rating']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)

for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

cosine_similarities = {}
for col in feature_columns:
    cosine_sim = cosine_similarity(list(movie_sample[col]))
    cosine_similarities[col] = cosine_sim

movie_ids = movie_sample['movieId'].tolist()
cosine_similarity_df = pd.DataFrame(index=movie_ids, columns=movie_ids)

for i in range(len(movie_ids)):
    for j in range(len(movie_ids)):
        similarities = []
        for col in feature_columns:
            similarities.append(cosine_similarities[col][i][j])
        cosine_similarity_df.at[movie_ids[i], movie_ids[j]] = sum(similarities) / len(similarities)

cosine_similarity_df = cosine_similarity_df.astype(float)

In [ ]:
# MinMax-Scaling
scaler = MinMaxScaler()
scaled_all = cosine_similarity_df.copy()
scale_col= list(cosine_similarity_df.columns)

scaled_all[scale_col] = scaler.fit_transform(scaled_all[scale_col])
scaled_all.head()

In [ ]:
scaled_all.to_csv(path+ '1210 CosSim_All.csv', encoding= 'utf-8', index= False)

### 3.2 Cosine - Title

In [ ]:
movie_sample= pd.read_csv(path+ "1210 all_pp.csv")
movie_sample= movie_sample[['movieId', 'title']]
movie_sample.head()

In [ ]:
# Cosine-Similarity
feature_columns = ['title']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)

for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

cosine_similarities = {}
for col in feature_columns:
    cosine_sim = cosine_similarity(list(movie_sample[col]))
    cosine_similarities[col] = cosine_sim

movie_ids = movie_sample['movieId'].tolist()
cosine_similarity_df = pd.DataFrame(index=movie_ids, columns=movie_ids)

for i in range(len(movie_ids)):
    for j in range(len(movie_ids)):
        similarities = []
        for col in feature_columns:
            similarities.append(cosine_similarities[col][i][j])
        cosine_similarity_df.at[movie_ids[i], movie_ids[j]] = sum(similarities) / len(similarities)

cosine_similarity_df = cosine_similarity_df.astype(float)

In [ ]:
# MinMax-scaling
scaler = MinMaxScaler()
scaled_title = cosine_similarity_df.copy()
scale_col= list(cosine_similarity_df.columns)

scaled_title[scale_col] = scaler.fit_transform(scaled_title[scale_col])
scaled_title.head()

In [ ]:
scaled_title.to_csv(path+ '1210 CosSim_Title.csv', encoding= 'utf-8', index= False)

### 3.3 Cosine - Genres

In [ ]:
movie_sample= pd.read_csv(path+ "1210 all_pp.csv")
movie_sample= movie_sample[['movieId', 'genres']]
movie_sample.head()

In [ ]:
# Cosine-Similarity
feature_columns = ['genres']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)

for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

cosine_similarities = {}
for col in feature_columns:
    cosine_sim = cosine_similarity(list(movie_sample[col]))
    cosine_similarities[col] = cosine_sim

movie_ids = movie_sample['movieId'].tolist()
cosine_similarity_df = pd.DataFrame(index=movie_ids, columns=movie_ids)

for i in range(len(movie_ids)):
    for j in range(len(movie_ids)):
        similarities = []
        for col in feature_columns:
            similarities.append(cosine_similarities[col][i][j])
        cosine_similarity_df.at[movie_ids[i], movie_ids[j]] = sum(similarities) / len(similarities)

cosine_similarity_df = cosine_similarity_df.astype(float)

In [ ]:
# MinMax-scaling
scaler = MinMaxScaler()
scaled_genres = cosine_similarity_df.copy()
scale_col= list(cosine_similarity_df.columns)

scaled_genres[scale_col] = scaler.fit_transform(scaled_genres[scale_col])
scaled_genres.head()

In [ ]:
scaled_genres.to_csv(path+ '1210 CosSim_Genres.csv', encoding= 'utf-8', index= False)

### 3.4 Cosine - Tag

In [ ]:
movie_sample= pd.read_csv(path+ "1210 all_pp.csv")
movie_sample= movie_sample[['movieId', 'tag']]
movie_sample.head()

In [ ]:
# Cosine-Similarity
feature_columns = ['tag']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)

for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

cosine_similarities = {}
for col in feature_columns:
    cosine_sim = cosine_similarity(list(movie_sample[col]))
    cosine_similarities[col] = cosine_sim

movie_ids = movie_sample['movieId'].tolist()
cosine_similarity_df = pd.DataFrame(index=movie_ids, columns=movie_ids)

for i in range(len(movie_ids)):
    for j in range(len(movie_ids)):
        similarities = []
        for col in feature_columns:
            similarities.append(cosine_similarities[col][i][j])
        cosine_similarity_df.at[movie_ids[i], movie_ids[j]] = sum(similarities) / len(similarities)

cosine_similarity_df = cosine_similarity_df.astype(float)

In [ ]:
# MinMax-scaling
scaler = MinMaxScaler()
scaled_tag = cosine_similarity_df.copy()
scale_col= list(cosine_similarity_df.columns)

scaled_tag[scale_col] = scaler.fit_transform(scaled_tag[scale_col])
scaled_tag.head()

In [ ]:
scaled_tag.to_csv(path+ '1210 CosSim_Tag.csv', encoding= 'utf-8', index= False)

### 3.5 Cosine - Storyline

In [ ]:
movie_sample= pd.read_csv(path+ "1210 all_pp.csv")
movie_sample= movie_sample[['movieId', 'storyline']]
movie_sample.head()

In [ ]:
# Cosine-Similarity
feature_columns = ['storyline']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)

for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

cosine_similarities = {}
for col in feature_columns:
    cosine_sim = cosine_similarity(list(movie_sample[col]))
    cosine_similarities[col] = cosine_sim

movie_ids = movie_sample['movieId'].tolist()
cosine_similarity_df = pd.DataFrame(index=movie_ids, columns=movie_ids)

for i in range(len(movie_ids)):
    for j in range(len(movie_ids)):
        similarities = []
        for col in feature_columns:
            similarities.append(cosine_similarities[col][i][j])
        cosine_similarity_df.at[movie_ids[i], movie_ids[j]] = sum(similarities) / len(similarities)

cosine_similarity_df = cosine_similarity_df.astype(float)

In [ ]:
# MinMax-scaling
scaler = MinMaxScaler()
scaled_story = cosine_similarity_df.copy()
scale_col= list(cosine_similarity_df.columns)

scaled_story[scale_col] = scaler.fit_transform(scaled_story[scale_col])
scaled_story.head()

In [ ]:
scaled_story.to_csv(path+ '1210 CosSim_Story.csv', encoding= 'utf-8', index= False)

## 4. pandas Euclidean-Similarity

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
import ast

path= "/content/drive/MyDrive/23 U 4-2/빅데이터처리및응용/Data/"

### 4.1 Euclidean - All

In [ ]:
movie_sample= pd.read_csv(path+ "1210 all_pp.csv")

In [ ]:
# Euclidean Similarity
feature_columns = ['title', 'era', 'genres', 'tag', 'storyline', 'runtime', 'continent', 'age_rating']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)
for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

def euclidean_similarity(arr1, arr2):
    return np.linalg.norm(arr1 - arr2)
euclidean_similarities = {col: np.zeros((len(movie_sample), len(movie_sample))) for col in feature_columns}

for col in feature_columns:
    for i in range(len(movie_sample)):
        for j in range(len(movie_sample)):
            euc_sim = euclidean_similarity(movie_sample[col][i], movie_sample[col][j])
            euclidean_similarities[col][i][j] = euc_sim

movie_ids = movie_sample['movieId'].tolist()
euclidean_similarity_df = pd.DataFrame(index=movie_ids, columns=movie_ids)

for i in range(len(movie_ids)):
    for j in range(len(movie_ids)):
        similarities = []
        for col in feature_columns:
            similarities.append(euclidean_similarities[col][i][j])
        euclidean_similarity_df.at[movie_ids[i], movie_ids[j]] = sum(similarities) / len(similarities)

euclidean_similarity_df = euclidean_similarity_df.astype(float)

In [ ]:
# MinMax-Scaling
scaler = MinMaxScaler()
scaled_all = euclidean_similarity_df.copy()
scale_col= list(euclidean_similarity_df.columns)

scaled_all[scale_col] = scaler.fit_transform(scaled_all[scale_col])
scaled_all.head()

In [ ]:
scaled_all.to_csv(path+ '1210 EucSim_All.csv', encoding= 'utf-8', index= False)

### 4.2 Euclidean - Title

In [ ]:
movie_sample= pd.read_csv(path+ "1210 all_pp.csv")
movie_sample= movie_sample[['movieId', 'title']]
movie_sample.head()

In [ ]:
# Euclidean Similarity
feature_columns = ['title']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)
for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

def euclidean_similarity(arr1, arr2):
    return np.linalg.norm(arr1 - arr2)
euclidean_similarities = {col: np.zeros((len(movie_sample), len(movie_sample))) for col in feature_columns}

for col in feature_columns:
    for i in range(len(movie_sample)):
        for j in range(len(movie_sample)):
            euc_sim = euclidean_similarity(movie_sample[col][i], movie_sample[col][j])
            euclidean_similarities[col][i][j] = euc_sim

movie_ids = movie_sample['movieId'].tolist()
euclidean_similarity_df = pd.DataFrame(index=movie_ids, columns=movie_ids)

for i in range(len(movie_ids)):
    for j in range(len(movie_ids)):
        similarities = []
        for col in feature_columns:
            similarities.append(euclidean_similarities[col][i][j])
        euclidean_similarity_df.at[movie_ids[i], movie_ids[j]] = sum(similarities) / len(similarities)

euclidean_similarity_df = euclidean_similarity_df.astype(float)

In [ ]:
# MinMax-Scaling
scaler = MinMaxScaler()
scaled_title = euclidean_similarity_df.copy()
scale_col= list(euclidean_similarity_df.columns)

scaled_title[scale_col] = scaler.fit_transform(scaled_title[scale_col])
scaled_title.head()

In [ ]:
scaled_title.to_csv(path+ '1210 EucSim_Title.csv', encoding= 'utf-8', index= False)

### 4.3 Euclidean - Genres

In [ ]:
movie_sample= pd.read_csv(path+ "1210 all_pp.csv")
movie_sample= movie_sample[['movieId', 'genres']]
movie_sample.head()

In [ ]:
# Euclidean Similarity
feature_columns = ['genres']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)
for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

def euclidean_similarity(arr1, arr2):
    return np.linalg.norm(arr1 - arr2)
euclidean_similarities = {col: np.zeros((len(movie_sample), len(movie_sample))) for col in feature_columns}

for col in feature_columns:
    for i in range(len(movie_sample)):
        for j in range(len(movie_sample)):
            euc_sim = euclidean_similarity(movie_sample[col][i], movie_sample[col][j])
            euclidean_similarities[col][i][j] = euc_sim

movie_ids = movie_sample['movieId'].tolist()
euclidean_similarity_df = pd.DataFrame(index=movie_ids, columns=movie_ids)

for i in range(len(movie_ids)):
    for j in range(len(movie_ids)):
        similarities = []
        for col in feature_columns:
            similarities.append(euclidean_similarities[col][i][j])
        euclidean_similarity_df.at[movie_ids[i], movie_ids[j]] = sum(similarities) / len(similarities)

euclidean_similarity_df = euclidean_similarity_df.astype(float)

In [ ]:
# MinMax-Scaling
scaler = MinMaxScaler()
scaled_genres = euclidean_similarity_df.copy()
scale_col= list(euclidean_similarity_df.columns)

scaled_genres[scale_col] = scaler.fit_transform(scaled_genres[scale_col])
scaled_genres.head()

In [ ]:
scaled_genres.to_csv(path+ '1210 EucSim_Genres.csv', encoding= 'utf-8', index= False)

### 4.4 Euclidean - Tag

In [ ]:
movie_sample= pd.read_csv(path+ "1210 all_pp.csv")
movie_sample= movie_sample[['movieId', 'tag']]
movie_sample.head()

In [ ]:
# Euclidean Similarity
feature_columns = ['tag']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)
for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

def euclidean_similarity(arr1, arr2):
    return np.linalg.norm(arr1 - arr2)
euclidean_similarities = {col: np.zeros((len(movie_sample), len(movie_sample))) for col in feature_columns}

for col in feature_columns:
    for i in range(len(movie_sample)):
        for j in range(len(movie_sample)):
            euc_sim = euclidean_similarity(movie_sample[col][i], movie_sample[col][j])
            euclidean_similarities[col][i][j] = euc_sim

movie_ids = movie_sample['movieId'].tolist()
euclidean_similarity_df = pd.DataFrame(index=movie_ids, columns=movie_ids)

for i in range(len(movie_ids)):
    for j in range(len(movie_ids)):
        similarities = []
        for col in feature_columns:
            similarities.append(euclidean_similarities[col][i][j])
        euclidean_similarity_df.at[movie_ids[i], movie_ids[j]] = sum(similarities) / len(similarities)

euclidean_similarity_df = euclidean_similarity_df.astype(float)

In [ ]:
# MinMax-Scaling
scaler = MinMaxScaler()
scaled_tag = euclidean_similarity_df.copy()
scale_col= list(euclidean_similarity_df.columns)

scaled_tag[scale_col] = scaler.fit_transform(scaled_tag[scale_col])
scaled_tag.head()

In [ ]:
scaled_tag.to_csv(path+ '1210 EucSim_Tag.csv', encoding= 'utf-8', index= False)

### 4.5 Euclidean - Storyline

In [ ]:
movie_sample= pd.read_csv(path+ "1210 all_pp.csv")
movie_sample= movie_sample[['movieId', 'storyline']]
movie_sample.head()

In [ ]:
# Euclidean Similarity
feature_columns = ['storyline']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)
for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

def euclidean_similarity(arr1, arr2):
    return np.linalg.norm(arr1 - arr2)
euclidean_similarities = {col: np.zeros((len(movie_sample), len(movie_sample))) for col in feature_columns}

for col in feature_columns:
    for i in range(len(movie_sample)):
        for j in range(len(movie_sample)):
            euc_sim = euclidean_similarity(movie_sample[col][i], movie_sample[col][j])
            euclidean_similarities[col][i][j] = euc_sim

movie_ids = movie_sample['movieId'].tolist()
euclidean_similarity_df = pd.DataFrame(index=movie_ids, columns=movie_ids)

for i in range(len(movie_ids)):
    for j in range(len(movie_ids)):
        similarities = []
        for col in feature_columns:
            similarities.append(euclidean_similarities[col][i][j])
        euclidean_similarity_df.at[movie_ids[i], movie_ids[j]] = sum(similarities) / len(similarities)

euclidean_similarity_df = euclidean_similarity_df.astype(float)

In [ ]:
# MinMax-Scaling
scaler = MinMaxScaler()
scaled_story = euclidean_similarity_df.copy()
scale_col= list(euclidean_similarity_df.columns)

scaled_story[scale_col] = scaler.fit_transform(scaled_story[scale_col])
scaled_story.head()

In [ ]:
scaled_story.to_csv(path+ '1210 EucSim_Story.csv', encoding= 'utf-8', index= False)